# VIEW DATA
This notebook views and inspects raw NYC TLC Yellow taxi data from January 2021


In [1]:
import pandas as pd

## VIEW January's raw data


In [2]:
df1 = pd.read_parquet("../raw/yellow_tripdata_2021-01.parquet")
print("Raw data from January has shape:", df1.shape)


Raw data from January has shape: (1369769, 19)


In [3]:
print("First ten rows of the data: ")
df1.head(10)

First ten rows of the data: 


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2021-01-01 00:30:10,2021-01-01 00:36:12,1.0,2.10,1.0,N,142,43,2,8.0,3.0,0.5,0.00,0.0,0.3,11.80,2.5,NaN
1,1,2021-01-01 00:51:20,2021-01-01 00:52:19,1.0,0.20,1.0,N,238,151,2,3.0,0.5,0.5,0.00,0.0,0.3,4.30,0.0,NaN
2,1,2021-01-01 00:43:30,2021-01-01 01:11:06,1.0,14.70,1.0,N,132,165,1,42.0,0.5,0.5,8.65,0.0,0.3,51.95,0.0,NaN
3,1,2021-01-01 00:15:48,2021-01-01 00:31:01,0.0,10.60,1.0,N,138,132,1,29.0,0.5,0.5,6.05,0.0,0.3,36.35,0.0,NaN
4,2,2021-01-01 00:31:49,2021-01-01 00:48:21,1.0,4.94,1.0,N,68,33,1,16.5,0.5,0.5,4.06,0.0,0.3,24.36,2.5,NaN
5,1,2021-01-01 00:16:29,2021-01-01 00:24:30,1.0,1.60,1.0,N,224,68,1,8.0,3.0,0.5,2.35,0.0,0.3,14.15,2.5,NaN
6,1,2021-01-01 00:00:28,2021-01-01 00:17:28,1.0,4.10,1.0,N,95,157,2,16.0,0.5,0.5,0.00,0.0,0.3,17.30,0.0,NaN
7,1,2021-01-01 00:12:29,2021-01-01 00:30:34,1.0,5.70,1.0,N,90,40,2,18.0,3.0,0.5,0.00,0.0,0.3,21.80,2.5,NaN
8,1,2021-01-01 00:39:16,2021-01-01 01:00:13,1.0,9.10,1.0,N,97,129,4,27.5,0.5,0.5,0.00,0.0,0.3,28.80,0.0,NaN
9,1,2021-01-01 00:26:12,2021-01-01 00:39:46,2.0,2.70,1.0,N,263,142,1,12.0,3.0,0.5,3.15,0.0,0.3,18.95,2.5,NaN


#### Findings from `.info()`

* **Dtypes:** Data types are mostly correct. `tpep_pickup_datetime` and `tpep_dropoff_datetime` are already proper `datetime64[ns]` objects.
* **Null Values:**
    * `airport_fee`: Almost 100% null (only 5 non-null values)
    * `passenger_count`, `RatecodeID`, `store_and_fwd_flag`, `congestion_surcharge`: These columns share the same number of missing values (~98k, or 7% of the data). 

-> Action: This needs to be handled in the QA step.

In [4]:
print("General information about columns, non-null count, and type of data of a month data:")
df1.info()

General information about columns, non-null count, and type of data of a month data:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1369769 entries, 0 to 1369768
Data columns (total 19 columns):
 #   Column                 Non-Null Count    Dtype         
---  ------                 --------------    -----         
 0   VendorID               1369769 non-null  int64         
 1   tpep_pickup_datetime   1369769 non-null  datetime64[ns]
 2   tpep_dropoff_datetime  1369769 non-null  datetime64[ns]
 3   passenger_count        1271417 non-null  float64       
 4   trip_distance          1369769 non-null  float64       
 5   RatecodeID             1271417 non-null  float64       
 6   store_and_fwd_flag     1271417 non-null  object        
 7   PULocationID           1369769 non-null  int64         
 8   DOLocationID           1369769 non-null  int64         
 9   payment_type           1369769 non-null  int64         
 10  fare_amount            1369769 non-null  float64       
 11  extr

#### Findings from `.describe()`

This reveals several anomalies and outliers that will define our QA Rules:
1.  **`passenger_count`**: `min = 0.0`. (An invalid value -> Rule: Must be > 0).
2.  **`trip_distance`**: `min = 0.0` and `max` is extremely large (263k). (Invalid values -> Rule: Must be > 0 and below a reasonable threshold).
3.  **`fare_amount`**: `min = -490.0`. (Invalid value -> Rule: Must be > 0).
4.  **`total_amount`**: `min = -492.8`. (Invalid value -> Rule: Must be > 0).

In [5]:
print("Info about numerical columns in data:")
df1.describe()

Info about numerical columns in data:


,VendorID,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
count,1.369769e+06,1.271417e+06,1.369769e+06,1.271417e+06,1.369769e+06,1.369769e+06,1.369769e+06,1.369769e+06,1.369769e+06,1.369769e+06,1.369769e+06,1.369769e+06,1.369769e+06,1.369769e+06,1.271417e+06,5.0
mean,1.721725e+00,1.411508e+00,4.631983e+00,1.035081e+00,1.652474e+02,1.614957e+02,1.188578e+00,1.209663e+01,9.705133e-01,4.930412e-01,1.918098e+00,2.477473e-01,2.969412e-01,1.747440e+01,2.239047e+00,0.0
std,5.925347e-01,1.059831e+00,3.939037e+02,5.994830e-01,6.783854e+01,7.210795e+01,5.776546e-01,1.291337e+01,1.231258e+00,7.632059e-02,2.597151e+00,1.672761e+00,4.222168e-02,1.469342e+01,7.989435e-01,0.0
min,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,-4.900000e+02,-5.500000e+00,-5.000000e-01,-1.000000e+02,-3.112000e+01,-3.000000e-01,-4.928000e+02,-2.500000e+00,0.0
25%,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.240000e+02,1.070000e+02,1.000000e+00,6.000000e+00,0.000000e+00,5.000000e-01,0.000000e+00,0.000000e+00,3.000000e-01,1.080000e+01,2.500000e+00,0.0
50%,2.000000e+00,1.000000e+00,1.700000e+00,1.000000e+00,1.620000e+02,1.620000e+02,1.000000e+00,8.500000e+00,0.000000e+00,5.000000e-01,1.860000e+00,0.000000e+00,3.000000e-01,1.380000e+01,2.500000e+00,0.0
75%,2.000000e+00,1.000000e+00,3.020000e+00,1.000000e+00,2.360000e+02,2.360000e+02,1.000000e+00,1.350000e+01,2.500000e+00,5.000000e-01,2.750000e+00,0.000000e+00,3.000000e-01,1.912000e+01,2.500000e+00,0.0
max,6.000000e+00,8.000000e+00,2.631633e+05,9.900000e+01,2.650000e+02,2.650000e+02,4.000000e+00,6.960500e+03,8.250000e+00,5.000000e-01,1.140440e+03,8.117500e+02,3.000000e-01,7.661280e+03,3.000000e+00,0.0


In [6]:
print("Columns of a month data are:")
df1.columns

Columns of a month data are:


Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee'],
      dtype='object')

## VIEW DATA TABLE ZONE

In [7]:
tzl = pd.read_csv("../raw/taxi_zone_lookup.csv")
print(tzl.shape)
tzl.head(5)

(265, 4)


,LocationID,Borough,Zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone


In [8]:
tzl.describe()

,LocationID
count,265.000000
mean,133.000000
std,76.643112
min,1.000000
25%,67.000000
50%,133.000000
75%,199.000000
max,265.000000


In [9]:
tzl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 265 entries, 0 to 264
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   LocationID    265 non-null    int64 
 1   Borough       264 non-null    object
 2   Zone          264 non-null    object
 3   service_zone  263 non-null    object
dtypes: int64(1), object(3)
memory usage: 8.4+ KB


## VIEW DATA OF SOME SPECIFIC COLUMNS


In [10]:
df1.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee'],
      dtype='object')

In [11]:
print("Values in column VendorID are: ")
df1['VendorID'].value_counts()

Values in column VendorID are: 


2    937141
1    422337
6     10291
Name: VendorID, dtype: int64

In [12]:
print("Values in column Payment type are:")
df1['payment_type'].value_counts()

Values in column Payment type are:


1    934475
2    322891
0     98352
3      8384
4      5667
Name: payment_type, dtype: int64

In [13]:
print("Statistic info of Passenger count column is: ")
df1['passenger_count'].describe()

Statistic info of Passenger count column is: 


count    1.271417e+06
mean     1.411508e+00
std      1.059831e+00
min      0.000000e+00
25%      1.000000e+00
50%      1.000000e+00
75%      1.000000e+00
max      8.000000e+00
Name: passenger_count, dtype: float64

In [14]:
print("Statistic info of Extra column is: ")
df1['extra'].describe()

Statistic info of Extra column is: 


count    1.369769e+06
mean     9.705133e-01
std      1.231258e+00
min     -5.500000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      2.500000e+00
max      8.250000e+00
Name: extra, dtype: float64

In [15]:
print("Unique values in Airport fee column are:")
list(df1['airport_fee'].unique())

Unique values in Airport fee column are:


[nan, 0.0]

In [16]:
print("The number of non-null values and null values(NA) in column Store and fwd flag is:")
df1['store_and_fwd_flag'].notna().value_counts()

The number of non-null values and null values(NA) in column Store and fwd flag is:


True     1271417
False      98352
Name: store_and_fwd_flag, dtype: int64